Start this first
```bash
$ litex_server --uart --uart-port /dev/ttyUSB0 --uart-baudrate 1152000
```

In [4]:
%pylab notebook
# from struct import pack, unpack
# import time
# from scipy.signal import *
# import sys
sys.path.append("..")
from common import *

Populating the interactive namespace from numpy and matplotlib


In [89]:
class HMC_SPI:
    # config bit offsets
    OFFLINE = 0      # all pins high-z (reset=1)
    CS_POLARITY = 3  # active level of chip select (reset=0)
    CLK_POLARITY = 4 # idle level of clk (reset=0)
    CLK_PHASE = 5    # first edge after cs assertion to sample data on (reset=0)
    LSB_FIRST = 6    # LSB is the first bit on the wire (reset=0)
    HALF_DUPLEX = 7  # 3-wire SPI, in/out on mosi (reset=0)
    DIV_READ = 16    # SPI read clk divider (reset=0)
    DIV_WRITE = 24   # f_clk / f_spi_write == div_write + 2
    # xfer bit offsets
    CS_MASK = 0      # Active high bit mask of chip selects to assert (reset=0)
    WRITE_LENGTH = 16  # How many bits to write and ...
    READ_LENGTH = 24  # when to switch over in half duplex mode

    def __init__(self, r):
        self.r = r
        self.cs_mask = 0x0002
        
        r.regs.spi_config.write(
            (1 << HMC_SPI.HALF_DUPLEX) |
            (0xFF << HMC_SPI.DIV_WRITE) |
            (0xFF << HMC_SPI.DIV_READ)
        )

    def set_reg(self, adr, val):
        # 16 bit write + 8 bit write transfer (3 wire SPI)
        r.regs.spi_xfer.write(
            (self.cs_mask << HMC_SPI.CS_MASK) |
            (16 + 8 << HMC_SPI.WRITE_LENGTH) |
            (0 << HMC_SPI.READ_LENGTH)
        )
        # R/W + W1 + W0 + A[13] + D[8]
        word = (0 << 23) | ((adr & 0x1FFF) << 8) | (val & 0xFF)
        # Send 24 bit / 32 bit starting from MSB
        word <<= 8
        self.r.regs.spi_mosi_data.write(word)
        self.r.regs.spi_start.write(1)

    def get_reg(self, adr):
        # 16 bit write + 8 bit read transfer (3 wire SPI)
        r.regs.spi_xfer.write(
            (self.cs_mask << HMC_SPI.CS_MASK) |
            (16 << HMC_SPI.WRITE_LENGTH) |
            (8 << HMC_SPI.READ_LENGTH)
        )
        word = (1 << 23) | ((adr & 0x1FFF) << 8)
        # Send 24 bit / 32 bit starting from MSB
        word <<= 8
        self.r.regs.spi_mosi_data.write(word)
        self.r.regs.spi_start.write(1)
        return self.r.regs.spi_miso_data.read() & 0xFF

# Connec to Zedboard PL

In [8]:
r = conLitexServer()

Connected to Port 1234
HMC7044 spi bvridge 2019-09-06 17:06:35


In [57]:
hex(r.regs.ctrl_scratch.read())

'0xdeadbeef'

In [11]:
r.regs.ctrl_scratch.write(0xDEADBEEF)

# Setup SPI

In [90]:
s = HMC_SPI(r)

In [83]:
s.set_reg(0, 1)

In [91]:
s.get_reg(0x79)

255

In [47]:
while True:
    s.get_reg(0x78)
    time.sleep(1)

KeyboardInterrupt: 

In [92]:
r.close()